In [1]:
import leafmap
import pystac_client
import planetary_computer
import rioxarray as rio
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
from shapely.ops import linemerge
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from riverrem.REMMaker import REMMaker
import osmnx.features as oxf
from shapely import Point, box
import pandas as pd
from geocube.api.core import make_geocube
from matplotlib.colors import LogNorm

In [ ]:
m = leafmap.Map()
m

In [ ]:
aoi_gdf = gpd.GeoDataFrame.from_features(m.draw_features, crs=4326)
aoi_gdf = aoi_gdf.to_crs(3857).to_crs(4326)
aoi = box(*aoi_gdf.total_bounds)
prj = ccrs.epsg(aoi_gdf.estimate_utm_crs().to_epsg())

In [ ]:
## get contours in aoi from copernicus global DEM hosted on planetary computer
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

search = catalog.search(collections=['cop-dem-glo-30'],
                        intersects=aoi.envelope)
items = list(search.get_items())

# if aoi crosses multiple DEM tiles stitch 'em together
if len(items) > 1:
    data = []
    for item in items:
        signed_asset = planetary_computer.sign(item.assets["data"])
        with rio.open_rasterio(signed_asset.href) as f:
            data.append(f.squeeze().drop_vars("band").rename('z'))
    dem = ((xr.merge(data))
           .rio.set_crs(4326)
           .rio.write_transform()
           .rio.clip_box(*aoi.bounds))['z']
    f.close()
else:
    signed_asset = planetary_computer.sign(items[0].assets["data"])
    dem =  (rio.open_rasterio(signed_asset.href)
            .squeeze()
            .rio.set_crs(4326)
            .rio.write_transform()
            .rio.clip_box(*aoi.bounds)).rename('z')

In [ ]:
dem_prj = dem.rio.reproject(prj, resolution=30, nodata=np.nan)
fig, ax = plt.subplots(subplot_kw={'projection':prj})
dem_prj.plot(ax=ax)
dem_prj.rio.to_raster('tmp.tif')

In [ ]:
from riverrem.REMMaker import REMMaker
# provide the DEM file path and desired output directory
rem_maker = REMMaker(dem='tmp.tif', out_dir='.')
# create an REM
rem_maker.make_rem()

In [ ]:
rem = rio.open_rasterio('tmp_REM.tif')

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection':prj},
                       figsize=[12,12])

((rem + 1 + np.abs(rem.min(skipna=True)))).plot(robust=True,
                                                           cmap='Blues',
                                                           add_colorbar=False,
                                                           ax=ax)

ax.set_facecolor('k')

In [ ]:
river_points = gpd.read_file('tmp_river_pts.shp')
river_buffer = gpd.GeoDataFrame(geometry=[river_points.dissolve('id').buffer(500)[1]], 
                                data={'val':[1]},
                                crs=prj)
for_clipping = make_geocube(river_buffer, resolution=30)['val'].rio.reproject_match(rem)
clipped = xr.where(for_clipping==1, rem, np.nan)